# Object Detection with OpenCV
Object detection, as implement here, will not only categorize what is in an image is
but also where the object is as well by placing bounding box around the object.

For object detection, we'll be using deep learning using a Single Shot Detectors and MobileNets.
When combined together these methods can be used for super fast, real-time object detection
on a resource constrained device like the Raspberry Pi.

Research this
* [Basic motion detection and tracking with Python and OpenCV](http://www.pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/)
* [Object Tracking using OpenCV (C++/Python)](https://www.learnopencv.com/object-tracking-using-opencv-cpp-python/)
* [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/)
* [YOLO Object Detection with OpenCV](https://www.pyimagesearch.com/2018/11/12/yolo-object-detection-with-opencv/)
* [In-Browser object detection using YOLO and TensorFlow.js](https://towardsdatascience.com/in-browser-object-detection-using-yolo-and-tensorflow-js-d2a2b7429f7c)

`python deep_learning_object_detection.py --image images/example_01.jpg --prototxt MobileNetSSD_deploy.prototxt.txt --model MobileNetSSD_deploy.caffemodel`

In [1]:
# import the necessary packages
import numpy as np
import time
import cv2

# minimum probability to filter weak detections
min_confidence = 0.2

# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car",
           "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
           "pottedplant", "sheep","sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe("/home/jeff/Jupyter-Notebooks/src/object-detection-deep-learning/MobileNetSSD_deploy.prototxt.txt",
                               "/home/jeff/Jupyter-Notebooks/src/object-detection-deep-learning/MobileNetSSD_deploy.caffemodel")

[INFO] loading model...


In [2]:
# load the input image and construct an input blob for the image
# by resizing to a fixed 300x300 pixels and then normalizing it
# (note: normalization is done via the authors of the MobileNet SSD
# implementation)
#image = cv2.imread("/home/jeff/Jupyter-Notebooks/src/object-detection-deep-learning/images/example_03.jpg")
#image = cv2.imread("/home/jeff/Pictures/Lenna.jpg")
#image = cv2.imread("/home/jeff/Pictures/eagle.png")
#image = cv2.imread("/home/jeff/Pictures/jemma.png")
image = cv2.imread("/home/jeff/Pictures/people-walking-above-1.png")

(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(image, 0.007843, (300, 300), 127.5)

# pass the blob through the network and obtain the detections and predictions
net.setInput(blob)
start = time.time()
detections = net.forward()
end = time.time()
print("[INFO] computing object detections {:.5} seconds".format(end - start))

[INFO] computing object detections 0.073041 seconds


In [3]:
# loop over the detections
start = time.time()
for i in np.arange(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the prediction
    confidence = detections[0, 0, i, 2]

    # filter out weak detections by ensuring the `confidence` is
    # greater than the minimum confidence
    if confidence > min_confidence:
        # extract the index of the class label from the `detections`,
        # then compute the (x, y)-coordinates of the bounding box for
        # the object
        idx = int(detections[0, 0, i, 1])
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # display the prediction
        label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
        print("[INFO] {}".format(label))
        cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx], 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
end = time.time()
print("[INFO] detection labeling took {:.5} seconds".format(end - start))

[INFO] bird: 74.14%
[INFO] detection labeling took 0.0010839 seconds


In [4]:
# show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)

-1

## Sources
* [Object detection with deep learning and OpenCV](http://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/)
* [Real-time object detection with deep learning and OpenCV](http://www.pyimagesearch.com/2017/09/18/real-time-object-detection-with-deep-learning-and-opencv/)